# Azure ML Inference Pipeline for HIFIS-RNN-MLP
This notebook defines an Azure machine learning pipeline for batch inference and submits the pipeline as an experiment to be run on an Azure virtual machine. It then publishes the pipeline in the workspace and schedules it to be run at a regular interval.

In [ ]:
# Import statements
import azureml.core
from azureml.core import Experiment
from azureml.core import Workspace, Datastore
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.core import Pipeline, PipelineRun
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.environment import Environment
from azureml.core.runconfig import RunConfiguration
import shutil

### Register the workspace and configure its Python environment.

In [ ]:
# Get reference to the workspace
ws = Workspace.from_config("./ws_config.json")

# Set workspace's environment
env = Environment.from_pip_requirements(name = "HIFIS_env", file_path = "./../requirements.txt")
env.python.conda_dependencies.add_pip_package("azureml-core")
env.python.conda_dependencies.add_pip_package("sendgrid")
env.register(workspace=ws)
runconfig = RunConfiguration(conda_dependencies=env.python.conda_dependencies)
print(env.python.conda_dependencies.serialize_to_string())

# Move AML ignore file to root folder
aml_ignore_path = shutil.copy('./.amlignore', './../.amlignore') 

### Create references to persistent and intermediate data
Create DataReference objects that point to our raw data on the blob. Configure a PipelineData object to point to preprocessed data stored on the blob. Pipeline data is intermediate, meaning that it is produced by a step and will be fed as input to a subsequent step.

In [ ]:
# Get the blob datastores associated with this workspace
hifis_blob_ds = Datastore(ws, name='hifisrnnmlp_ds')
raw_data_blob_ds = Datastore(ws, name='hifis_raw_ds')

# Create data references to folders on the blobs
raw_data_dr = DataReference(
    datastore=raw_data_blob_ds,
    data_reference_name="raw_data",
    path_on_datastore="hifis/")
inference_dr = DataReference(
    datastore=hifis_blob_ds,
    data_reference_name="inference",
    path_on_datastore="inference/")

# Set up intermediate pipeline data to store preprocessed data and serialized objects needed for inference (e.g. explainer)
preprocess_pd = PipelineData(
    "preprocessed_output",
    datastore=hifis_blob_ds,
    output_name="preprocessed_output",
    output_mode="mount")

### Compute Target
Specify and configure the compute target for this workspace. If a compute cluster by the name we specified does not exist, create a new compute cluster.

In [ ]:
# Define some constants
CT_NAME = "ds3v2-infer"          # Name of our compute cluster
VM_SIZE = "STANDARD_DS3_V2"      # Specify the Azure VM for execution of our pipeline
MIN_NODES = 1                    # Min number of compute nodes in cluster
MAX_NODES = 3                    # Max number of compute nodes in cluster

# Set up the compute target for this experiment
try:
    compute_target = AmlCompute(ws, CT_NAME)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating new compute target")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=VM_SIZE, min_nodes=MIN_NODES, max_nodes=MAX_NODES)    
    compute_target = ComputeTarget.create(ws, CT_NAME, provisioning_config)  # Create the compute cluster
    
    # Wait for cluster to be provisioned
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20) 
    
print("Azure Machine Learning Compute attached")
print("Compute targets: ", ws.compute_targets)
compute_target = ws.compute_targets[CT_NAME]

### Define pipeline and submit experiment.
Define the steps of an Azure machine learning pipeline. Create an Azure Experiment that will run our pipeline. Submit the experiment to the execution environment.

In [ ]:
# Define preprocessing step in the ML pipeline
step1 = PythonScriptStep(name="preprocess_step",
                         script_name="azure/preprocess_step/preprocess_step.py",
                         arguments=["--rawdatadir", raw_data_dr, "--inferencedir", inference_dr, "--preprocessedoutputdir", 
                                    preprocess_pd],
                         inputs=[raw_data_dr, inference_dr],
                         outputs=[preprocess_pd],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         params={"PIPELINE": "inference"},
                         allow_reuse=False)

# Define batch inference step in the ML pipeline
step2 = PythonScriptStep(name="inference_step",
                         script_name="azure/inference_step/inference_step.py",
                         arguments=["--preprocessedoutputdir", preprocess_pd, "--inferencedir", inference_dr],
                         inputs=[preprocess_pd, inference_dr],
                         outputs=[],
                         compute_target=compute_target, 
                         source_directory="./../",
                         runconfig=runconfig,
                         allow_reuse=False)


# Construct the ML pipeline from the steps
steps = [step1, step2]
inference_pipeline = Pipeline(workspace=ws, steps=steps)
inference_pipeline.validate()

# Define a new experiment and submit a new pipeline run to the compute target.
experiment = Experiment(workspace=ws, name='BatchInferenceExperiment_v1')
inference_run = experiment.submit(inference_pipeline, regenerate_outputs=False)
print("Pipeline is submitted for execution")

# Move AML ignore file back to original folder
aml_ignore_path = shutil.move(aml_ignore_path, './.amlignore') 

### Publish the pipeline
Wait for the pipeline run to finish. Then publish the pipeline. The pipeline will be visible as an endpoint in the Pipelines tab in the workspace on Azure Machine Learning studio. Delete the training compute cluster to prevent further cost.

In [ ]:
# Wait for the pipeline to finish running.
inference_run.wait_for_completion()

# Publish the pipeline.
published_pipeline = inference_run.publish_pipeline(
     name="HIFIS-RNN-MLP Inference Pipeline",
     description="Azure ML Pipeline that runs model inference on all clients in HIFIS database and computes LIME explanations",
     version="1.0")

### Schedule the pipeline
Schedule the published pipeline to run weekly.

In [ ]:
recurrence = ScheduleRecurrence(frequency="Week", interval=1)
pipeline_id = "inference-pipeline"
recurring_schedule = Schedule.create(ws, name="WeeklyInferenceSchedule", 
                            description="Scheduled weekly batch inference",
                            pipeline_id=published_pipeline.id, 
                            experiment_name=experiment.name, 
                            recurrence=recurrence)

### Disable scheduled pipelines
Disable all scheduled pipelines

In [ ]:
schedules = Schedule.list(ws)
for schedule in schedules:
    print(schedule)
    schedule.disable()